In [40]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Extract Data

In [2]:
from bs4 import BeautifulSoup 
entries = []
entry = []
urlnumber = 2
while urlnumber < 105: 
    url = 'https://forums.edmunds.com/discussion/7526/general/x/midsize-sedans-2-0/p%d' % (urlnumber,)
    try:
        r = requests.get(url, timeout = 10) 
    except Exception as e:
        print("Error message:",e)
        break;

    data = r.text
    
    soup = BeautifulSoup(data, 'lxml')
    
    for div in soup.find_all('div'):
        entry = []
        if(div.get('class') != None and div.get('class')[0] == 'Comment'): 
            ps = div.find_all('p') 
            aas = div.find_all('a')
            spans = div.find_all('span')
            times = div.find_all('time') 
            concat_str = ''
            for str in aas[1].contents: 
                if str != "<br>" or str != "<br/>": 
                    concat_str = (concat_str + ' '+ str).encode("utf-8").strip()
            entry.append(concat_str)

            concat_str = ''
            for str in times[0].contents:
                if str != "<br>" or str != "<br/>":
                    concat_str = (concat_str + ' '+ str).encode('iso-8859-1').strip()
            entry.append(concat_str)

            for div in div.find_all('div'):
                if (div.get('class') != None and div.get('class')[0] == 'Message'): 
                    blockquotes = []
                    x = div.get_text()
                    for bl in div.find_all('blockquote'):
                        blockquotes.append(bl.get_text()) 
                        bl.decompose()
                    ascii_encoding = div.get_text().replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                    latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                    entry.append(latin1_encoding)

                    for bl in blockquotes:
                        ascii_encoding = bl.replace("\n"," ").replace("<br/>","").encode('ascii','replace')
                        latin1_encoding = ascii_encoding.decode('ascii').encode('iso-8859-1')
                        entry.append(latin1_encoding)

            entries.append(entry)
            
    urlnumber += 1

stringlist=[[x.decode('iso-8859-1') for x in entry] for entry in entries]
with open('edmunds_extraction.csv', 'w') as output:
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerows(stringlist)

print ("Wrote to edmunds_extraction.csv")

Wrote to edmunds_extraction.csv


In [54]:
import pandas as pd

In [55]:
df=pd.read_csv('edmunds_extraction.csv',header=None)

In [56]:
df.columns=['userid','date','message']

In [57]:
df

,userid,date,message
0,elroy5,April 2007,"No, I have not driven a Sonata. I have driven..."
1,plekto,April 2007,Of course the Buick LaCrosse qualifies. It ce...
2,punkr77,April 2007,One thing that drops the Maxima off my shoppi...
3,backy,April 2007,Good point. Another way Nissan differentiates...
4,lilengineerboy,April 2007,"Elroy,Autocross is typically a ""course"" confi..."
...,...,...,...
5145,urnews,September 2007,"That is one bad (as in good) looking Camry, e..."
5146,tlong,September 2007,2. The majority of G6s I see are not rentals....
5147,elroy5,September 2007,"And what if someone likes car B better, shoul..."
5148,thegraduate,September 2007,Question for the audience: Which manufacturer...


### Brands and models from external source plus models.csv file given

In [58]:
df_brands=pd.read_csv('brands.csv')
df_models=pd.read_csv('model.csv')
df_brand_model=pd.merge(df_models,df_brands,left_on='brand_id',right_on='id')
df_brand_model=df_brand_model[['name_x','name_y']].rename(columns={'name_x':'Model','name_y':'Brand'})
df_brand_model_given=pd.read_csv('models.csv',header=None).rename(columns={0:'Brand',1:'Model'})
df_brand_model['Model']=df_brand_model['Model'].apply(lambda x:x.lower())
df_brand_model['Brand']=df_brand_model['Brand'].apply(lambda x:x.lower())
df_brand_model_given['Model']=df_brand_model_given['Model'].apply(lambda x:x.lower())
df_brand_model_given['Brand']=df_brand_model_given['Brand'].apply(lambda x:x.lower())
df_brand_model_final=df_brand_model_given.append(df_brand_model).drop_duplicates()
df_brand_model=df_brand_model_final.reset_index().drop(columns='index')
df_brand_model

,Brand,Model
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx
...,...,...
1029,volvo,c30
1030,volvo,s40
1031,volvo,s70
1032,volvo,v50


In [59]:
df['message']=df['message'].astype('str')

In [60]:
#df=df[1:50]

### Stopwords removal

In [61]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['message'] = df['message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df)

              userid            date  \
0             elroy5      April 2007   
1             plekto      April 2007   
2            punkr77      April 2007   
3              backy      April 2007   
4     lilengineerboy      April 2007   
...              ...             ...   
5145          urnews  September 2007   
5146           tlong  September 2007   
5147          elroy5  September 2007   
5148     thegraduate  September 2007   
5149         m1miata  September 2007   

                                                message  
0     No, I driven Sonata. I driven Camrys, I've hea...  
1     Of course Buick LaCrosse qualifies. It certain...  
2     One thing drops Maxima shopping list (but matt...  
3     Good point. Another way Nissan differentiates ...  
4     Elroy,Autocross typically "course" configured ...  
...                                                 ...  
5145  That one bad (as good) looking Camry, even tho...  
5146  2. The majority G6s I see rentals. You easily ...

### Tokenization

In [62]:
import nltk
from nltk.tokenize import word_tokenize
df['message']=df['message'].apply(lambda x:word_tokenize(x))

### Tagging words with pos_tag and lemmatization
- Extracted only the nouns from pos tag as brands and models will come under them
- Didn't use porter stemmer as it can manipulate the brand names

In [63]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
df['message']=df['message'].apply(lambda x:nltk.pos_tag(x))

In [64]:
for l in range(len(df['message'])):
    df['message'].iloc[l]=[j[0] for j in df['message'].iloc[l] if j[1].startswith("NN")]

In [65]:
df['message']

0       [Sonata, Camrys, Sonata, Camry, dynamics, Opti...
1       [course, Buick, LaCrosse, qualifies, luxury, c...
2       [thing, Maxima, list, %, buyers, option, Altim...
3                    [point, way, Nissan, Altima, Maxima]
4       [Elroy, Autocross, course, bunch, traffic, con...
                              ...                        
5145                                [good, Camry, Toyota]
5146    [majority, G6s, rentals, rentals, wheels, stic...
5147    [someone, likes, car, B, better, differences, ...
5148    [Question, audience, manufacturer, dominates, ...
5149    [Look, OK, rest, New, Malibu, photos, Hard, da...
Name: message, Length: 5150, dtype: object

In [66]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w).lower() for w in text]
df['message']=df['message'].apply(lambda x:lemmatize_text(x))

from nltk.stem import PorterStemmer
ps = PorterStemmer()
def porter_stemmer(text):
    return [ps.stem(w).lower() for w in text]
#df['message'].apply(lambda x:porter_stemmer(x))

In [67]:
df

,userid,date,message
0,elroy5,April 2007,"[sonata, camrys, sonata, camry, dynamic, optim..."
1,plekto,April 2007,"[course, buick, lacrosse, qualifies, luxury, c..."
2,punkr77,April 2007,"[thing, maxima, list, %, buyer, option, altima..."
3,backy,April 2007,"[point, way, nissan, altima, maxima]"
4,lilengineerboy,April 2007,"[elroy, autocross, course, bunch, traffic, con..."
...,...,...,...
5145,urnews,September 2007,"[good, camry, toyota]"
5146,tlong,September 2007,"[majority, g6s, rental, rental, wheel, sticker..."
5147,elroy5,September 2007,"[someone, like, car, b, better, difference, pe..."
5148,thegraduate,September 2007,"[question, audience, manufacturer, dominates, ..."


In [68]:
df['message']

0       [sonata, camrys, sonata, camry, dynamic, optim...
1       [course, buick, lacrosse, qualifies, luxury, c...
2       [thing, maxima, list, %, buyer, option, altima...
3                    [point, way, nissan, altima, maxima]
4       [elroy, autocross, course, bunch, traffic, con...
                              ...                        
5145                                [good, camry, toyota]
5146    [majority, g6s, rental, rental, wheel, sticker...
5147    [someone, like, car, b, better, difference, pe...
5148    [question, audience, manufacturer, dominates, ...
5149    [look, ok, rest, new, malibu, photo, hard, day...
Name: message, Length: 5150, dtype: object

In [69]:
df_brand_model

,Brand,Model
0,acura,integra
1,acura,legend
2,acura,vigor
3,acura,rlx
4,acura,ilx
...,...,...
1029,volvo,c30
1030,volvo,s40
1031,volvo,s70
1032,volvo,v50


In [70]:
for i in range(len(df['message'])):
    for j in range(len(df_brand_model)):
        if(df_brand_model['Model'].iloc[j] in df['message'].iloc[i]):
            print(df_brand_model['Model'].iloc[j])
            idx=df['message'].iloc[i].index(df_brand_model['Model'].iloc[j])
            df['message'].iloc[i][idx]=df_brand_model['Brand'].iloc[j]

sonata
optima
sentra
camry
camrys
chance
lacrosse
regal
accord
altima
maxima
altima
maxima
civic
accords
sentra
california
laguna
impala
f-150
camry
corolla
accord
toyotas
vw
soul
camry
fusion
accord
camry
taurus
fords
century
explorer
mustang
fusion
accord
fords
fusion
fords
fusions
accord
accords
mazda6
accord
fusion
country
fusion
mazda6
comfort
accord
idea
soul
fusion
mazda6
soul
fusion
taurus
escort
corolla
corolla
cars
malibu
cruiser
accord
comfort
deville
soul
cars
malibu
cruiser
accord
comfort
deville
soul
fusion
accord
accords
civic
accord
accords
altima
camry
accord
camry
cars
comfort
fusion
accord
camry
camry
accord
accords
focus
civic
fusion
accord
camry
chance
explorer
fusion
fords
pilot
accord
hondas
accord
accord
hondas
sebring
ex-l
accord
optima
altima
camry
accord
camry
fusion
f-150
mondeo
fusion
accord
mazda6
altima
comfort
insight
one
sebring
sonata
altima
legacy
one
sonata
sonata
fusion
accord
sonata
legacy
sebring
impala
fusion
accord
hondas
accords
sonata
altima
a

avalon
camry
fusion
city
accord
altima
camry
fusion
sonata
milan
mercury
fusion
accord
sonata
mazda6
lx
mustang
fusion
accord
fusion
accord
sonata
mazda6
lx
accord
sonata
fusion
accord
altima
accord
sonata
sonata
sonata
optima
sonata
optima
partner
accord
fusion
accord
accords
chance
fusion
accord
fusion
fusions
accord
vw
fusion
sonata
taurus
fusion
taurus
accord
idea
fusion
taurus
sonata
accord
fusion
taurus
fusions
camry
fusion
fusion
accord
cars
lx
accord
sonata
fords
hondas
toyotas
malibu
fusion
fords
accord
hondas
aura
toyotas
cts
fusion
accord
sonata
altima
sonata
fusion
taurus
accord
sonata
accord
accords
accord
mazda6
fusion
accord
altima
accords
sonata
santa
sonatas
mazda6
aura
camry
fusion
accord
problems
fusion
city
accord
altima
fusion
taurus
escape
accord
altima
fusion
focus
accord
accord
fusion
sonata
idea
fusion
fusion
taurus
fusions
sonata
sonatas
taurus
fusion
fusion
taurus
note
impala
malibu
aura
yaris
cts
camry
coupe
note
accord
fusion
accord
altima
taurus
mazda6
fus

altima
altima
accord
accord
city
frontier
accord
sonata
mazda6
legacy
gt
fusion
cr-v
prius
aztek
prius
accord
idea
fusion
fusions
accord
accords
sonata
camrys
accord
country
accord
malibu
fusion
accord
sonata
altima
fusion
accord
sonata
mazda6
altima
camry
fusion
accord
sonata
accord
accord
sonata
accord
comfort
accord
hondas
accord
accord
camry
lx
accord
sonata
accord
accords
country
mustang
fusion
fusions
accord
accords
mx-5
vw
glc
country
accord
idea
accord
accord
accord
accords
note
accord
camry
explorer
fusion
accord
mazda6
mazda6
fusion
accord
camry
chance
accord
lx
fusion
taurus
accord
chance
mercedes-benz
fusion
accord
lx
taurus
accord
accords
camry
fusion
accord
altima
fusion
accord
accord
idea
fusion
accord
camry
fusion
accord
sebring
accord
iq
lx
sonata
iq
sonata
elantra
optima
mercedes-benz
mercury
iq
civic
accord
lx
fusion
accord
accords
mazda6
mazda6i
civic
mercury
vw
accord
altima
camry
leaf
vw
civic
sonata
accord
accord
camry
accords
camrys
altima
maxima
accord
camry
fu

odyssey
accord
fusion
odyssey
accord
odyssey
accord
fusion
f-150
fusions
accord
sonata
odyssey
ex-l
accord
fusion
camry
fusion
accord
sonata
camry
accord
fusion
accord
camry
fusion
camry
accord
legacy
camry
gt
fusion
fusion
zephyr
mazda6
mazda6
mustang
gt
accords
accord
fusion
fusion
fusions
fusion
accord
pony
camry
accord
maxima
gt
fusion
ex-l
accord
sonata
mustang
mustang
explorer
mustang
mazda6
fusion
accord
accords
accord
altima
fusion
accord
altima
odyssey
fusion
camry
city
accord
fusion
accord
altima
camry
altima
fusion
accord
sonata
camry
fusions
accord
odyssey
accord
accords
accord
coupe
mustang
gt
california
gt
gt
fusion
fusion
accord
camry
civic
accord
legacy
legacy
outback
legacy
lacrosse
lx
fusion
taurus
azera
fusion
sonata
aura
camry
camrys
chance
aura
camry
lacrosse
impala
impala
azera
maxima
avalon
taurus
mkz
fusion
taurus
accord
fusion
accord
milan
mercury
altima
aura
camry
fusion
accord
fusion
aura
aura
fusion
aura
note
aura
impala
g8
accord
aura
fusion
focus
aura
note

fords
accord
one
fusion
fusion
mazda6
altima
camry
fusion
accord
aura
camry
fusion
aura
lx
fusion
city
accord
sonata
optima
continental
stratus
fusions
fusion
vibe
aura
malibu
fusion
fusions
aura
fusion
city
accord
accords
maxima
aura
camry
vectra
wrx
corvette
malibu
accord
g6
aura
cts
altima
altimas
aura
one
country
fusion
aura
malibu
fusion
altima
aura
note
country
fusion
accord
sonata
aura
camry
accord
sonata
aura
camry
aura
fusion
altima
avalon
fusion
fusion
aura
jetta
fusion
accord
sonata
milan
mercury
altima
aura
fusion
comfort
city
mazda6
milan
mercury
altima
neon
fusion
fusions
accord
kias
fusion
accord
camry
odyssey
tundra
fusion
accord
milan
mercury
altimas
fusion
milan
mercury
malibu
aura
camry
accord
aura
trailblazer
malibu
g6
chance
idea
odyssey
civic
excel
mustang
aura
gt
one
note
chance
focus
frontier
malibu
cts
tundra
altima
malibu
altima
one
fusion
fusion
taurus
legacy
quattro
a4
altima
altima
malibu
altima
chance
neon
altima
legacy
comfort
gt
cars
accord
accords
altim

accord
sonata
accord
sonata
optima
accords
malibu
accord
sonata
altima
aura
camry
passat
slk
optima
cr-v
accord
accords
mazda6
legacy
gt
fusion
city
accord
mazda6
coupe
focus
civic
accord
camry
a6
ridgeline
accord
sonata
passat
accord
sonata
accord
accords
accords
camry
accord
camry
lx
accord
racer
coupe
coupe
accord
accords
mazda6
accord
accord
accords
sonata
accord
camry
mazda6
accord
accord
altima
passat
jetta
vw
accord
accords
camry
accord
accord
accord
accords
accord
city
accord
taurus
accord
avalon
impala
malibu
aveo
mustang
taurus
accord
aura
g
aura
impala
accord
accord
idea
city
soul
city
city
accord
city
accord
accords
galant
optima
galant
taurus
city
passat
vw
jetta
vw
california
capital
lacrosse
accord
sonata
azera
altima
maxima
sedans
avalon
camry
passat
vws
passat
taurus
move
country
transporter
altima
passat
accord
accord
sonata
sedans
corolla
move
corvette
soul
sedans
accord
jetta
vw
vw
taurus
accord
racer
soul
taurus
city
accord
camry
s80
corvette
accord
sonata
camry
ta

mazda6
accord
accords
mazda6
mazda6
accord
altima
camry
altima
camry
impala
accord
camry
camrys
impala
malibu
chevy
f150
camry
camry
camry
fusion
accord
sonata
altima
tundra
camry
camry
camry
aura
accord
camry
accord
tundra
camry
malibu
sonata
mazda6
camry
move
accord
accords
one
fusion
accord
aura
accord
country
lx
accord
impala
fusion
accord
mazda6
altima
comfort
camry
pilot
mazda6
asx
malibu
fusion
accord
aura
lx
accord
idea
accord
camry
camry
camry
malibu
fusion
accord
sonata
altima
camry
accord
camry
camry
sonata
aura
camry
fusion
accord
lx
accord
camry
accord
altima
one
elantra
altima
accord
altima
camry
accord
elantra
altima
accord
sonata
altima
camry
accord
sonata
wave
accord
sonata
sonata
optima
accord
sonata
optima
optima
altima
camry
accord
camry
accord
accords
sonata
aura
one
sonata
aura
accord
sonata
fusion
sonata
tundra
camry
cts
elantra
sonata
azera
sonatas
camry
lx
civic
accord
azera
accord
silverado
sonata
santa
sonatas
optima
camry
camrys
one
corvette
impala
malibu
ct

In [74]:
list(df_brand_model['Model'])

['integra',
 'legend',
 'vigor',
 'rlx',
 'ilx',
 'mdx',
 'rdx',
 'tsx',
 'rsx',
 'quattro',
 'a4',
 'a6',
 'a5',
 'a7',
 'q3',
 'q5',
 'q7',
 'a8',
 'a3',
 '3-series',
 '5-series',
 '6-series',
 '4-series',
 'xdrive',
 'alpina',
 '330ci',
 '328i',
 '330c',
 '335i',
 '335d',
 '528i',
 '535i',
 '128i',
 '640i',
 '3er',
 '5er',
 'm5',
 'm3',
 'm6',
 'bmws',
 'bmwesque',
 'park avenue',
 'la crosse',
 'lacrosse',
 'century',
 'enclave',
 'encore',
 'regal',
 'buicks',
 'escalade',
 'srx',
 'xts',
 'cars',
 'grand blazer',
 'trailblazer',
 'silverado',
 'colorado',
 'corvette',
 'suburban',
 'cavalier',
 'caprice',
 'captiva',
 'equinox',
 'express',
 'lacetti',
 'montana',
 'uplander',
 'orlando',
 'tornado',
 'tracker',
 'traverse',
 'camaro',
 'cobalt',
 'impala',
 'lumina',
 'malibu',
 'prisma',
 'tavera',
 'agile',
 'celta',
 'cruze',
 'lanos',
 'omega',
 'optra',
 'sonic',
 'spark',
 'tahoe',
 'aveo',
 'lova',
 'n200',
 'niva',
 'onix',
 's-10',
 'sail',
 'volt',
 'chevy',
 'malibus'

In [71]:
df_brand_model['Brand']=df_brand_model['Brand'].str.replace(',','')
df_brand_model['Model']=df_brand_model['Model'].str.replace('.','')
df_brand_model['Brand']=df_brand_model['Brand'].str.replace('.','')
df_brand_model['Model']=df_brand_model['Model'].str.replace(',','')
df_brand_model['Brand']=df_brand_model['Brand'].apply(lambda x:x.strip())
df_brand_model['Model']=df_brand_model['Model'].apply(lambda x:x.strip())

In [78]:
for brand in set(df_brand_model['Brand']):
    df[brand]=np.nan

In [79]:
df

,userid,date,message,ford,problem,dodge,skoda,citroen,hyundai,alfa romeo,...,lancia,buick,bentley,dfm,infiniti,audi,acura,honda,volvo,lamborghini
0,elroy5,April 2007,"[hyundai, toyota, sonata, toyota, dynamic, kia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plekto,April 2007,"[course, buick, buick, qualifies, luxury, car,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,punkr77,April 2007,"[thing, nissan, list, %, buyer, option, nissan...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,backy,April 2007,"[point, way, nissan, nissan, nissan]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,lilengineerboy,April 2007,"[elroy, autocross, course, bunch, traffic, con...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5145,urnews,September 2007,"[good, toyota, toyota]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5146,tlong,September 2007,"[majority, g6s, rental, rental, wheel, sticker...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5147,elroy5,September 2007,"[someone, like, car, b, better, difference, pe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5148,thegraduate,September 2007,"[question, audience, manufacturer, dominates, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
for brand in set(df_brand_model['Brand']):
    for j in range(len(df['message'])):
        if(brand in df['message'].iloc[j]):
            df[brand].iloc[j]=1
        else:
            df[brand].iloc[j]=0

In [82]:
df.iloc[:, 3:].sum(axis=1)

### Top 10 brands by frequency

In [112]:
df_top_ten_brands=pd.DataFrame(df.iloc[:, 3:].sum(axis=0)).reset_index()
df_top_ten_brands.columns=['brand','frequency']
df_top_ten_brands=df_top_ten_brands.sort_values(by='frequency',ascending=False).reset_index().drop(columns=['index'])

In [116]:
list(df_top_ten_brands['brand'][2:12])

['honda',
 'ford',
 'toyota',
 'hyundai',
 'nissan',
 'mazda',
 'sedan',
 'problem',
 'chevrolet',
 'chrysler']

In [95]:
df[df['sum']>0]

,userid,date,message,ford,problem,dodge,skoda,citroen,hyundai,alfa romeo,...,buick,bentley,dfm,infiniti,audi,acura,honda,volvo,lamborghini,sum
0,elroy5,April 2007,"[hyundai, toyota, sonata, toyota, dynamic, kia...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
1,plekto,April 2007,"[course, buick, buick, qualifies, luxury, car,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0
2,punkr77,April 2007,"[thing, nissan, list, %, buyer, option, nissan...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,backy,April 2007,"[point, way, nissan, nissan, nissan]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,lilengineerboy,April 2007,"[elroy, autocross, course, bunch, traffic, con...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5145,urnews,September 2007,"[good, toyota, toyota]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5146,tlong,September 2007,"[majority, g6s, rental, rental, wheel, sticker...",0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5147,elroy5,September 2007,"[someone, like, car, b, better, difference, pe...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5148,thegraduate,September 2007,"[question, audience, manufacturer, dominates, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0


### Lift Similarity of top 10 brands

In [175]:
df_1=df.iloc[:,3:]

In [176]:
df_1=df_1[['honda',
 'ford',
 'toyota',
 'hyundai',
 'nissan',
 'mazda',
 'sedan',
 'problem',
 'chevrolet',
 'chrysler']]

In [177]:
df_1

,honda,ford,toyota,hyundai,nissan,mazda,sedan,problem,chevrolet,chrysler
0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5145,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5148,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [178]:
#df_1=df_1.drop(columns=['sum'])

In [179]:
#df_1=df_1[df_1.columns[df_1.sum()>0]]

In [180]:
#len(df_1[(df_1[df_1.columns[0]]==1) & (df_1[df_1.columns[1]]==1)])/len(df_1[(df_1[df_1.columns[1]]==1)])

In [181]:
df_1=df_1.reset_index().drop(columns='index')

In [182]:
df_1

,honda,ford,toyota,hyundai,nissan,mazda,sedan,problem,chevrolet,chrysler
0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5145,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5148,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [186]:
idx=0
for i in range(len(df_1.columns)):
    for j in range(i+1,len(df_1.columns)):
        #df_1[df_1.columns[i+1]].iloc[idx]=99
        print(i,j,df_1.columns[i],df_1.columns[j],len(df_1[(df_1[df_1.columns[i]]==1) & (df_1[df_1.columns[j]]==1)])*len(df_1)/(len(df_1[(df_1[df_1.columns[i]]==1)])*len(df_1[(df_1[df_1.columns[j]]==1)])))
        df_1[df_1.columns[j]].iloc[idx]=len(df_1[(df_1[df_1.columns[i]]==1) & (df_1[df_1.columns[j]]==1)])*len(df_1)/(len(df_1[(df_1[df_1.columns[i]]==1)])*len(df_1[(df_1[df_1.columns[j]]==1)]))
    idx=idx+1
    

0 1 honda ford 1.1636635870212513
0 2 honda toyota 1.6880830323770362
0 3 honda hyundai 1.4774968621122466
0 4 honda nissan 1.409758005152742
0 5 honda mazda 1.1830681395898788
0 6 honda sedan 1.2509530967474893
0 7 honda problem 1.22854728117886
0 8 honda chevrolet 1.3300129127467257
0 9 honda chrysler 1.3890451821486305
1 2 ford toyota 1.368277499452378
1 3 ford hyundai 1.2128153155791965
1 4 ford nissan 1.2138193099965127
1 5 ford mazda 1.1917512958849044
1 6 ford sedan 1.0803090251722491
1 7 ford problem 1.2860161352285824
1 8 ford chevrolet 1.3657249382975372
1 9 ford chrysler 1.3826663310680427
2 3 toyota hyundai 1.8885603340175492
2 4 toyota nissan 2.102607367099872
2 5 toyota mazda 0.9212298497411507
2 6 toyota sedan 1.436098220095173
2 7 toyota problem 1.1516165373097151
2 8 toyota chevrolet 2.169938782758885
2 9 toyota chrysler 1.260695886325956
3 4 hyundai nissan 1.67784434944297
3 5 hyundai mazda 1.1081838695419546
3 6 hyundai sedan 1.4713124151852672
3 7 hyundai problem 0.

In [187]:
df_1=df_1[:len(df_1.columns)]

In [188]:
cols=df_1.columns

In [189]:
m,n = df_1.shape
df_1=pd.DataFrame(np.where(np.arange(m)[:,None] >= np.arange(n),np.nan,df_1))
df_1=df_1.fillna(0)

In [190]:
df_1.index=cols
df_1.columns=cols

In [191]:
df_1

,honda,ford,toyota,hyundai,nissan,mazda,sedan,problem,chevrolet,chrysler
honda,0.0,1.163664,1.688083,1.477497,1.409758,1.183068,1.250953,1.228547,1.330013,1.389045
ford,0.0,0.000000,1.368277,1.212815,1.213819,1.191751,1.080309,1.286016,1.365725,1.382666
toyota,0.0,0.000000,0.000000,1.888560,2.102607,0.921230,1.436098,1.151617,2.169939,1.260696
hyundai,0.0,0.000000,0.000000,0.000000,1.677844,1.108184,1.471312,0.821811,1.575978,1.425250
nissan,0.0,0.000000,0.000000,0.000000,0.000000,1.542101,1.714441,1.120052,2.020929,1.141509
mazda,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.704256,1.063914,0.906123,0.679174
sedan,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.410630,1.698104,1.340099
problem,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.924167,1.384688
chevrolet,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.638381
chrysler,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### 1/Lift(dissimilarity)

In [193]:
df_2=pd.DataFrame()
for l in df_1.columns:
    df_2[l]=np.nan
    df_2[l]=df_1[l].astype('int')
    df_2[l]=1/df_1[l]

In [194]:
df_2.columns

Index(['honda', 'ford', 'toyota', 'hyundai', 'nissan', 'mazda', 'sedan',
       'problem', 'chevrolet', 'chrysler'],
      dtype='object')

In [195]:
df_2

,honda,ford,toyota,hyundai,nissan,mazda,sedan,problem,chevrolet,chrysler
honda,inf,0.859355,0.592388,0.676820,0.709342,0.845260,0.799390,0.813969,0.751872,0.719919
ford,inf,inf,0.730846,0.824528,0.823846,0.839101,0.925661,0.777595,0.732212,0.723240
toyota,inf,inf,inf,0.529504,0.475600,1.085505,0.696331,0.868345,0.460842,0.793213
hyundai,inf,inf,inf,inf,0.596003,0.902377,0.679665,1.216825,0.634526,0.701631
nissan,inf,inf,inf,inf,inf,0.648466,0.583281,0.892816,0.494822,0.876033
mazda,inf,inf,inf,inf,inf,inf,0.586766,0.939925,1.103603,1.472376
sedan,inf,inf,inf,inf,inf,inf,inf,0.708903,0.588892,0.746214
problem,inf,inf,inf,inf,inf,inf,inf,inf,1.082056,0.722184
chevrolet,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.610359
chrysler,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
